In [ ]:
import umap
from spacy.lang.de import German
from tqdm import tqdm
from spacy.lang.de.stop_words import STOP_WORDS
import pandas as pd
import json
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from wordcloud import WordCloud
import seaborn as sns
from gensim.models import Word2Vec
import numpy as np
from sklearn import metrics
from sklearn.decomposition import PCA

### Reading

In [ ]:
f = open("cleared_data.txt", "r", encoding="utf-8")
cleared_data_law = f.readlines()
f.close()

In [ ]:
f = open("cleared_data_labels.txt", "r", encoding="utf-8")
cleared_data_labels = f.readlines()
f.close()

In [ ]:
cleared_data = []
for var in cleared_data_law:
    temp = []
    for j in list(var.split(" ")):
        temp.append(j)
    cleared_data.append(temp)

In [ ]:
print(cleared_data[0])

### Word2Vec

In [ ]:
modell = Word2Vec(cleared_data,workers=5,iter=5)

In [ ]:
w1 = 'Nummer'
modell.wv.most_similar(positive=w1)

In [ ]:
print(modell)

In [ ]:
def avg(lst):
    suma=0
    for word in lst:
        suma = suma + modell[word]
    return suma/len(lst)

In [ ]:
cleared_data_vectors = []
for sentence in tqdm(cleared_data):
    fin_cleared_data.append(avg(sentence))

In [ ]:
Sum_of_squared_distances = []
K = range(2,10)
for k in K:
    print(k)
    km = KMeans(n_clusters=k, max_iter=200, n_init=1)
    km = km.fit(cleared_data_vectors)
    Sum_of_squared_distances.append(km.inertia_)
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
true_k = 5
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=200, n_init=1)
model.fit(X)
labels=model.labels_

In [ ]:
true_k = 3
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=200, n_init=1)
model.fit(X)
labels=model.labels_

In [ ]:
words = modell.wv.vocab.keys()
feature_names = list(words)

In [ ]:
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = feature_names
for i in range(true_k):
    print ("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print (' %s' % terms[ind]),
    print()

In [ ]:
result={'cluster':labels,'law':cleared_data_law}
result=pd.DataFrame(result)
for k in range(0,true_k):
    s=result[result.cluster==k]
    text=s['law'].str.cat(sep=' ')
    text=text.lower()
    text=' '.join([word for word in text.split()])
    wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(text)
    print('Cluster: {}'.format(k))
    print('Titles')
    titles=law_cl[law_cl.cluster==k]['category']         
    print(titles.to_string(index=False))
    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()

In [ ]:
from sklearn.metrics import pairwise_distances_argmin

def find_clusters(X, n_clusters, rseed=2):
    # 1. Randomly choose clusters
    rng = np.random.RandomState(rseed)
    i = rng.permutation(X.shape[0])[:n_clusters]
    centers = X[i]
    
    while True:
        # 2a. Assign labels based on closest center
        labels = pairwise_distances_argmin(X, centers)
        
        # 2b. Find new centers from means of points
        new_centers = np.array([X[labels == i].mean(0)
                                for i in range(n_clusters)])
        
        # 2c. Check for convergence
        if np.all(centers == new_centers):
            break
        centers = new_centers
    
    return centers, labels

centers, labelss = find_clusters(X, 4)
plt.scatter(X[:, 0], X[:, 1], c=labelss,
            s=50, cmap='viridis');

In [ ]:
random_state = 0
# reduce the features to 2D
pca = PCA(n_components=2, random_state=random_state)
reduced_features = pca.fit_transform(fin_cleared_data)

# reduce the cluster centers to 2D
reduced_cluster_centers = pca.transform(model.cluster_centers_)
plt.scatter(reduced_features[:,0], reduced_features[:,1], c=model.predict(fin_cleared_data))
plt.scatter(reduced_cluster_centers[:, 0], reduced_cluster_centers[:,1], marker='x', s=150, c='b')

In [ ]:
from yellowbrick.text import UMAPVisualizer
umap = UMAPVisualizer(labels=["category"], metric='cosine')
umap.fit(modell.wv.vectors)
umap.show()

In [ ]:
plt.scatter(standard_embedding[:, 0], standard_embedding[:, 1], c=labels, s=0.1, cmap='Spectral');

In [ ]:
from sklearn.datasets import fetch_openml


clusterable_embedding = umap.UMAP(
    n_neighbors=30,
    min_dist=0.0,
    n_components=2,
    random_state=20,
).fit_transform(modell.wv.vectors)
plt.scatter(clusterable_embedding[:, 0], clusterable_embedding[:, 1],
            c=mnist.target, s=0.1, cmap='Spectral');

In [ ]:
random_state = 0
# reduce the features to 2D
umap = umap.UMAP(n_components=2,random_state=random_state)

In [ ]:
standard_embedding = umap.fit_transform(modell.wv.vectors)

In [ ]:
# reduce the cluster centers to 2D
reduced_cluster_centerss = umap.transform(model.cluster_centers_)
#plt.scatter(standard_embedding[:,0], standard_embedding[:,1], c=model.predict(modell.wv.vectors))
plt.scatter(reduced_cluster_centerss[:, 0], reduced_cluster_centerss[:,1], marker='x', s=150, c='b')